In [1]:
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import queue
import threading
import time

import fundamentalanalysis as fa
import fa_mods as famods
import fa_utils

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [2]:
all_companies = fa.available_companies(api_key)
tradeable_companies = famods.tradable_companies(api_key)

In [3]:
exchanges = all_companies.groupby('exchangeShortName')['exchange'].agg('unique').reset_index().rename(columns={'exchangeShortName': 'short_name', 'unique': 'exchange'})

In [4]:
datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(all_companies, file)

with open(f'../data/all_tradable_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(tradeable_companies, file)

with open(f'../data/exchanges_{datestamp}.pkl', 'wb') as file:
    pickle.dump(exchanges, file)

In [5]:
pd.set_option('display.max_rows', None)
# display(exchanges)

exchange_targets = [
    "AMEX",    # NYSE ARCA
    "AMS",     # Amsterdam
    "XETRA",   # Deutsche Boerse Exchange, Frankfurt
    "CPH",     # Copenhagen
    "STO",     # Stockholm
    "HEL",     # Helsinki
    "OSE",     # Oslo
    "EURONEXT",
    "NASDAQ",
    "NYSE",
    "LSE",
    "TLV",     # Tel Aviv
    "TSX",     # Toronto
]

In [6]:
companies_on_exchange = {}

total_num_companies = 0

for exchange_short_name in exchange_targets:
    ec = all_companies[(all_companies.exchangeShortName == exchange_short_name)]
    companies_on_exchange[exchange_short_name] = ec

    ec_num = len(ec)
    total_num_companies += ec_num
    print(f"{exchange_short_name} has {ec_num} companies")

print(f"Gathering {total_num_companies} companies from {len(exchange_targets)} exchanges")

AMEX has 2752 companies
AMS has 270 companies
XETRA has 3016 companies
CPH has 161 companies
STO has 846 companies
HEL has 163 companies
OSE has 271 companies
EURONEXT has 1590 companies
NASDAQ has 10657 companies
NYSE has 4066 companies
LSE has 6540 companies
TLV has 320 companies
TSX has 2756 companies
Gathering 33408 companies from 13 exchanges


In [7]:
thread_lock = threading.Lock()

def get_exchange_data(exchange: str):
    datestamp = datetime.now().strftime('%y%m%d')
    exchange_companies = all_companies[(all_companies.exchangeShortName == exchange)]
    exchange_stocks = exchange_companies[(exchange_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

    symbols = exchange_stocks.symbol.tolist()
    stock_data = {}
    failed = []
    num_stocks = len(symbols)
    cnt = 0

    symbol_queue = queue.Queue()
    [symbol_queue.put(s) for s in symbols]
    symbol_queue.qsize()

    def worker():
        nonlocal cnt
        nonlocal stock_data
        nonlocal failed
        while True:

            time.sleep(1)
            
            with thread_lock:
                cnt += 1
                symbol = symbol_queue.get()        
                print(f'{exchange} [{cnt}/{num_stocks}]: getting {symbol} - f[{len(failed)}], s[{len(stock_data)}]')

            if symbol in stock_data:
                print(f"Already have {symbol}, skipping")
                continue

            try:
                data = fa_utils.get_ticker_mf_am_data(symbol, api_key)
                stock_data[symbol] = data
                print(f"{symbol} SUCCESS")
            except:
                print(f"{symbol} FAILED")
                failed.append(symbol)

            symbol_queue.task_done()

    for x in range(4):
        name = "Thread_" + str(x)
        t = threading.Thread(name=name,target=worker, daemon=True).start()

    symbol_queue.join()

    print(len(failed))
    print(len(stock_data))

    with open(f'../data/all_{exchange.lower()}_mf_am_{datestamp}.pkl', 'wb') as file:
        pickle.dump(stock_data, file)

    with open(f'../data/all_{exchange.lower()}_mf_am_{datestamp}_failed.pkl', 'wb') as file:
        pickle.dump(failed, file)

In [8]:
# symbol = "WBEV"
# data = fa_utils.get_ticker_mf_am_data(symbol, api_key)

In [9]:
exchange_short_name = exchange_targets[5]
print(exchange_short_name)

STO


In [10]:
get_exchange_data(exchange_short_name)

STO [1/846]: getting KAPIAB.ST - f[0], s[0]
STO [2/846]: getting SWED-A.ST - f[0], s[0]
STO [3/846]: getting NDA-SE.ST - f[0], s[0]
STO [4/846]: getting TELIA.ST - f[0], s[0]
TELIA.ST SUCCESS
NDA-SE.ST SUCCESS
KAPIAB.ST SUCCESS
SWED-A.ST SUCCESS
STO [5/846]: getting HEBA-B.ST - f[0], s[4]
STO [6/846]: getting VPLAY-B.ST - f[0], s[4]
STO [7/846]: getting 8TRA.ST - f[0], s[4]
STO [8/846]: getting SCIB.ST - f[0], s[4]
HEBA-B.ST SUCCESS
VPLAY-B.ST SUCCESS
8TRA.ST SUCCESS
SCIB.ST SUCCESS
STO [9/846]: getting APRNDR.ST - f[0], s[8]
STO [10/846]: getting RAKE.ST - f[0], s[8]
STO [11/846]: getting MEKO.ST - f[0], s[8]
STO [12/846]: getting AROS.ST - f[0], s[8]
APRNDR.ST SUCCESS
RAKE.ST SUCCESS
MEKO.ST SUCCESS
AROS.ST SUCCESS
STO [13/846]: getting ESSITY-A.ST - f[0], s[12]
STO [14/846]: getting STW.ST - f[0], s[12]
STO [15/846]: getting KINV-A.ST - f[0], s[12]
STO [16/846]: getting PDX.ST - f[0], s[12]
ESSITY-A.ST SUCCESS
STW.ST SUCCESS
KINV-A.ST SUCCESS
PDX.ST SUCCESS
STO [17/846]: getting TEL

In [11]:
# for exchange_short_name in exchange_targets:
#     get_exchange_data(exchange_short_name)

In [12]:
exchange_short_name = "AMEX"

datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_{exchange_short_name.lower()}_mf_am_{datestamp}.pkl', 'rb') as file:
    stock_data_2 = pickle.load(file)

with open(f'../data/all_{exchange_short_name.lower()}_mf_am_{datestamp}_failed.pkl', 'rb') as file:
    stock_data_2_failed = pickle.load(file)


print(len(stock_data_2))
print(len(stock_data_2_failed))
# display(stock_data_2)

272
7
